In [1]:
import pandas as pd
import codecs
import os, re, json

In [ ]:
# https://storage.googleapis.com/cluebenchmark/tasks/cmrc2018_public.zip

In [9]:
train = json.load(open('./cmrc2018_public/train.json'))
dev = json.load(open('./cmrc2018_public/dev.json'))

In [17]:
train['data'][0]['paragraphs'][0]

{'id': 'TRAIN_186',
 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。',
 'qas': [{'question': '范廷颂是什么时候被任为主教的？',
   'id': 'TRAIN_186_QUERY_0',
   'answers': [{'text': '1963年', 'answer_start': 30}]},
  {'question': '1990年，

In [21]:
from transformers import BertTokenizerFast, BertForQuestionAnswering

tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-bert-wwm-ext')
model = BertForQuestionAnswering.from_pretrained('hfl/chinese-bert-wwm-ext')

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chinese-

In [48]:
paragraphs = []
questions = []
start_positions = []
end_positions = []
for paragraph in train['data']:
    for qa in paragraph['paragraphs'][0]['qas']:
        paragraphs.append(paragraph['paragraphs'][0]['context'])
        questions.append(qa['question'])
        start_positions.append(qa['answers'][0]['answer_start'])
        end_positions.append(qa['answers'][0]['answer_start'] + len(qa['answers'][0]['text']))

In [49]:
train_encodings = tokenizer(paragraphs, questions, 
                            return_tensors='pt', truncation=True, padding=True,
                           max_length=512)

train_encodings['start_positions'] = [train_encodings.char_to_token(idx, x) if train_encodings.char_to_token(idx, x) != None else -1
                                      for idx, x in enumerate(start_positions)]
train_encodings['end_positions'] = [train_encodings.char_to_token(idx, x-1) if train_encodings.char_to_token(idx, x-1) != None else -1
                                    for idx, x in enumerate(end_positions)]

In [50]:
paragraphs = []
questions = []
start_positions = []
end_positions = []
for paragraph in dev['data']:
    for qa in paragraph['paragraphs'][0]['qas']:
        paragraphs.append(paragraph['paragraphs'][0]['context'])
        questions.append(qa['question'])
        start_positions.append(qa['answers'][0]['answer_start'])
        end_positions.append(qa['answers'][0]['answer_start'] + len(qa['answers'][0]['text']))

In [51]:
val_encodings = tokenizer(paragraphs, questions, 
                            return_tensors='pt', truncation=True, padding=True,
                           max_length=512)

val_encodings['start_positions'] = [val_encodings.char_to_token(idx, x) if val_encodings.char_to_token(idx, x) != None else -1
                                      for idx, x in enumerate(start_positions) ]
val_encodings['end_positions'] = [val_encodings.char_to_token(idx, x-1) if val_encodings.char_to_token(idx, x) != None else -1
                                    for idx, x in enumerate(end_positions)]

In [52]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
    
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)

In [53]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for idx, batch in enumerate(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, 
                        start_positions=start_positions, 
                        end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()

        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        acc1 = ( (start_pred == start_positions).sum() / len(start_pred) ).item()
        acc2 = ( (end_pred == end_positions).sum() / len(start_pred) ).item()

        if idx % 10 == 0:
            print(loss.item(), acc1, acc2)
            with codecs.open('log.log', 'a') as up:
                up.write('{3}\t{0}\t{1}\t{2}\n'.format(loss.item(), acc1, acc2, 
                                                       str(epoch) + '/' + str(idx) +'/'+ str(len(train_loader))))

model.eval()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


5.914609909057617 0.0 0.0
5.220417022705078 0.125 0.0
4.716152191162109 0.0 0.0
3.629622459411621 0.25 0.25
2.7960753440856934 0.375 0.25
3.017632007598877 0.5 0.375
2.131814479827881 0.25 0.625
2.1228792667388916 0.5 0.25
3.46903657913208 0.125 0.0
2.7174510955810547 0.25 0.25
3.371978521347046 0.25 0.25
2.2220964431762695 0.25 0.375
2.3144443035125732 0.5 0.5
2.6459059715270996 0.25 0.375
2.48060941696167 0.5 0.375
1.8796577453613281 0.375 0.5
2.3050951957702637 0.5 0.375
1.8356908559799194 0.625 0.5
1.479246973991394 0.625 0.375
1.6082086563110352 0.5 0.75
1.8967199325561523 0.625 0.375
1.5556164979934692 0.375 0.5
2.332493305206299 0.625 0.25
1.8171426057815552 0.5 0.75
2.13638973236084 0.5 0.375
2.7112174034118652 0.25 0.375
3.0005276203155518 0.375 0.25
1.93096923828125 0.375 0.5
1.0288045406341553 0.875 0.75
2.2864294052124023 0.375 0.125
1.6522210836410522 0.75 0.25
1.6973814964294434 0.625 0.75
2.4134974479675293 0.125 0.5
1.4316622018814087 0.5 0.625
2.1923255920410156 0.125 

KeyboardInterrupt: 

In [80]:
def predcit(doc, query):
    print(doc)
    print('提问：', query)
    item = tokenizer([doc, query], max_length=512, return_tensors='pt', truncation=True, padding=True,)
    with torch.no_grad():
        input_ids = torch.tensor(item['input_ids']).to(device).reshape(1, -1)
        attention_mask = torch.tensor(item['attention_mask']).to(device).reshape(1, -1)
        
        outputs = model(input_ids[:, :512], attention_mask[:, :512])
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
    
    try:
        start_pred = item.token_to_chars(0, start_pred)
        end_pred = item.token_to_chars(0, end_pred)
    except:
        return ''
    
    if start_pred.start > end_pred.end:
        return ''
    else:
        return doc[start_pred.start:end_pred.end]

In [81]:
dev['data'][0]

{'paragraphs': [{'id': 'DEV_0',
   'context': '《战国无双3》（）是由光荣和ω-force开发的战国无双系列的正统第三续作。本作以三大故事为主轴，分别是以武田信玄等人为主的《关东三国志》，织田信长等人为主的《战国三杰》，石田三成等人为主的《关原的年轻武者》，丰富游戏内的剧情。此部份专门介绍角色，欲知武器情报、奥义字或擅长攻击类型等，请至战国无双系列1.由于乡里大辅先生因故去世，不得不寻找其他声优接手。从猛将传 and Z开始。2.战国无双 编年史的原创男女主角亦有专属声优。此模式是任天堂游戏谜之村雨城改编的新增模式。本作中共有20张战场地图（不含村雨城），后来发行的猛将传再新增3张战场地图。但游戏内战役数量繁多，部分地图会有兼用的状况，战役虚实则是以光荣发行的2本「战国无双3 人物真书」内容为主，以下是相关介绍。（注：前方加☆者为猛将传新增关卡及地图。）合并本篇和猛将传的内容，村雨城模式剔除，战国史模式可直接游玩。主打两大模式「战史演武」&「争霸演武」。系列作品外传作品',
   'qas': [{'question': '《战国无双3》是由哪两个公司合作开发的？',
     'id': 'DEV_0_QUERY_0',
     'answers': [{'text': '光荣和ω-force', 'answer_start': 11},
      {'text': '光荣和ω-force', 'answer_start': 11},
      {'text': '光荣和ω-force', 'answer_start': 11}]},
    {'question': '男女主角亦有专属声优这一模式是由谁改编的？',
     'id': 'DEV_0_QUERY_1',
     'answers': [{'text': '村雨城', 'answer_start': 226},
      {'text': '村雨城', 'answer_start': 226},
      {'text': '任天堂游戏谜之村雨城', 'answer_start': 219}]},
    {'question': '战国史模式主打哪两个模式？',
     'id': 'DEV_0_QUERY_2',
     'answ

In [82]:
predcit(dev['data'][0]['paragraphs'][0]['context'],
       dev['data'][0]['paragraphs'][0]['qas'][0]['question'])

《战国无双3》（）是由光荣和ω-force开发的战国无双系列的正统第三续作。本作以三大故事为主轴，分别是以武田信玄等人为主的《关东三国志》，织田信长等人为主的《战国三杰》，石田三成等人为主的《关原的年轻武者》，丰富游戏内的剧情。此部份专门介绍角色，欲知武器情报、奥义字或擅长攻击类型等，请至战国无双系列1.由于乡里大辅先生因故去世，不得不寻找其他声优接手。从猛将传 and Z开始。2.战国无双 编年史的原创男女主角亦有专属声优。此模式是任天堂游戏谜之村雨城改编的新增模式。本作中共有20张战场地图（不含村雨城），后来发行的猛将传再新增3张战场地图。但游戏内战役数量繁多，部分地图会有兼用的状况，战役虚实则是以光荣发行的2本「战国无双3 人物真书」内容为主，以下是相关介绍。（注：前方加☆者为猛将传新增关卡及地图。）合并本篇和猛将传的内容，村雨城模式剔除，战国史模式可直接游玩。主打两大模式「战史演武」&「争霸演武」。系列作品外传作品
提问： 《战国无双3》是由哪两个公司合作开发的？


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


'光荣和ω-force'

In [83]:
predcit(dev['data'][0]['paragraphs'][0]['context'],
       dev['data'][0]['paragraphs'][0]['qas'][1]['question'])

《战国无双3》（）是由光荣和ω-force开发的战国无双系列的正统第三续作。本作以三大故事为主轴，分别是以武田信玄等人为主的《关东三国志》，织田信长等人为主的《战国三杰》，石田三成等人为主的《关原的年轻武者》，丰富游戏内的剧情。此部份专门介绍角色，欲知武器情报、奥义字或擅长攻击类型等，请至战国无双系列1.由于乡里大辅先生因故去世，不得不寻找其他声优接手。从猛将传 and Z开始。2.战国无双 编年史的原创男女主角亦有专属声优。此模式是任天堂游戏谜之村雨城改编的新增模式。本作中共有20张战场地图（不含村雨城），后来发行的猛将传再新增3张战场地图。但游戏内战役数量繁多，部分地图会有兼用的状况，战役虚实则是以光荣发行的2本「战国无双3 人物真书」内容为主，以下是相关介绍。（注：前方加☆者为猛将传新增关卡及地图。）合并本篇和猛将传的内容，村雨城模式剔除，战国史模式可直接游玩。主打两大模式「战史演武」&「争霸演武」。系列作品外传作品
提问： 男女主角亦有专属声优这一模式是由谁改编的？


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


'村雨城'